네이버 시가총액 (코스피500개, 코스닥300개)

In [4]:
import csv
import requests
import datetime
from bs4 import BeautifulSoup

def stock_data(columns, sosok):
    data = list()
    data = data + [sosok]
    code = list()    
    #fn_link = list()
    for column in columns:        
        data = data + [column.text.strip()]            
        a = column.find("a",{"class":"tltle"})

        if a:
            code = [a.attrs["href"][20:]]
            #code = ["https://finance.naver.com" + a.attrs["href"]] 
            code_str = str(code)[4:10]
            #fn_link = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code_str+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
            fn_link = "https://comp.fnguide.com/SVO/WooriRenewal/Snapshot.asp?pGB=12&gicode=A"+code_str
            #print(fn_link)
            data = data + code + [fn_link]

    return data[0:-1]



today = datetime.date.today().strftime('%y%m%d')
filename = today + "_sise_code.csv"

f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "구분,No,종목명,종목코드,fn링크,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE".split(",")
print(type(title))
writer.writerow(title)

for sosok in range(2):
    if sosok==0:
        last_page = 11
    else:
        last_page = 7    

    url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=" + str(sosok) + "&page="

    for page in range(1, last_page):
        res =requests.get(url + str(page))
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml")

        data_rows = soup.find("table", attrs={"class":"type_2"}).find("tbody").find_all("tr")
        for row in data_rows:
            columns = row.find_all("td")
            #print(columns)
            #print("*****")
            if len(columns) <= 1:
                continue        
            #data = [column.get_text().strip() for column in columns]
            #print(data)
            #writer.writerow(data)
            
            data = stock_data(columns, sosok)
            #print(type(data))
            #print(data)

            writer.writerow(data)

print('save')
f.close()


<class 'list'>
save


In [68]:
import pandas as pd
import re

pd.set_option('display.max_rows', None)

df = pd.read_csv('251206_sise_code.csv', encoding="euc-kr")

df['등락률2']=df['등락률'].str.replace('%','').astype(float)
df['전일비2']=df['전일비'].str.replace(r'\s+',' ', regex=True)

df2 = df.sort_values(by=['등락률2','현재가'], ascending=True)
df3 = df2.loc[:, ['구분','종목명','현재가','전일비2','등락률2','외국인비율','PER','ROE']]

df_pr_ch = df3.loc[((df3['등락률2'] > 10) | (df3['등락률2'] < -5)) & (df3['외국인비율'] > 5), ['구분','종목명','현재가','전일비2','등락률2','외국인비율','PER','ROE']]
#display(df_pr_ch)   

df_sname =df3.loc[df3['종목명'].str.contains('에코|삼성전|제일기획|이노션')]                                                                                                                                                                            
display(df_sname)   


,구분,종목명,현재가,전일비2,등락률2,외국인비율,PER,ROE
744,1,에코앤드림,"21,750",하락 450,-2.03,1.43,25.47,8.41
32,0,삼성전기,"263,000","하락 2,000",-0.75,38.12,29.51,8.16
162,0,제일기획,"22,800",하락 100,-0.44,22.92,14.38,15.12
782,1,에코마케팅,"10,760",상승 40,0.37,17.13,11.97,12.87
366,0,이노션,"18,690",상승 160,0.86,21.07,8.84,10.48
501,1,에코프로비엠,"159,700","상승 1,800",1.14,12.38,"4,990.62",-6.26
4,0,삼성전자우,"80,100","상승 1,100",1.39,76.94,16.63,NaN
117,0,에코프로머티,"60,000","상승 1,000",1.69,19.80,37.62,-5.56
662,1,에코프로에이치엔,"27,150",상승 600,2.26,7.67,32.71,10.59
0,0,삼성전자,"108,400","상승 3,300",3.14,52.21,22.51,9.03
